In [ ]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:

colors = [
    'grey', 'black',
    "#FF5733",  # Vibrant Red-Orange
    "#33FF57",  # Bright Green
    "#5733FF",  # Bright Blue
    "#F1C40F",  # Sun Yellow
    "#8E44AD",  # Violet
    "#FF33A1",  # Hot Pink
    "#FF8C33",  # Orange
    "#3498DB",   # Light Blue
    "#8C33FF",  # Purple
    "#33D9FF",  # Sky Blue
    "#E74C3C",  # Strong Red
    "#16A085",  # Teal
    "#1F77B4",  # Medium Blue
    "#33FF8C",  # Light Green
    "#F39C12",  # Golden Yellow
    "#D35400",  # Orange
    "#2ECC71",  # Emerald Green
    "#9B59B6",  # Amethyst Purple
    "#FF338C",  # Deep Pink
    "#FF6F33",  # Coral
]


In [ ]:
inputFilePath = os.path.join('..', 'results', 'andrade_10_316.json')
with open(inputFilePath, 'r') as file:
    dataDct = json.load(file)

In [ ]:
outputDf = pd.DataFrame(dataDct['output'])
for idx, vs in enumerate(dataDct['visitedSteps']):
    outputDf[f'cluster_id_{idx:04.0f}'] = dataDct['visitedSteps'][idx]
nVisitedSteps = len(dataDct['visitedSteps'])
outputDf.rename(columns={'cluster_id': f'cluster_id_{nVisitedSteps:04.0f}'}, inplace=True)
nVisitedSteps += 1

In [ ]:
outputDf

In [ ]:
outputDir = os.path.join('..', 'plots')
os.makedirs(outputDir, exist_ok=True)


def createPlot(vs):
    fig, ax = plt.subplots(figsize=(20,15))
    plt.xlabel('x')
    plt.ylabel('y', rotation=0)
    plt.xlim(0,40)
    plt.ylim(0,25)
    
    if vs >= 3:
        vs -= 3
        xs = outputDf['x']
        ys = outputDf['y']
        xsCore = outputDf[outputDf['is_core'] == 1]['x']
        ysCore = outputDf[outputDf['is_core'] == 1]['y']
        xsNonCore = outputDf[outputDf['is_core'] == 0]['x']
        ysNonCore = outputDf[outputDf['is_core'] == 0]['y']

        destXs = []
        destYs = []
        sourceXs = []
        sourceYs = []

        for idx, neighborList in enumerate(dataDct['graph']):
            sourceXs += [ xs[idx] ] * len(neighborList)
            sourceYs += [ ys[idx] ] * len(neighborList)
            destXs += [ xs[neighbor] for neighbor in neighborList ]
            destYs += [ ys[neighbor] for neighbor in neighborList ]
        destXs = np.array(destXs)
        destYs = np.array(destYs)
        sourceXs = np.array(sourceXs)
        sourceYs = np.array(sourceYs)

        plt.quiver(sourceXs, sourceYs, destXs - sourceXs, destYs - sourceYs, angles='xy', scale_units='xy', scale=1, color='lightgrey')

        clusterColName = f'cluster_id_{vs:04.0f}'
        for cl, grp in outputDf.groupby(clusterColName):
            xsCore = grp[grp['is_core'] == 1]['x']
            ysCore = grp[grp['is_core'] == 1]['y']
            xsNonCore = grp[grp['is_core'] == 0]['x']
            ysNonCore = grp[grp['is_core'] == 0]['y']

            plt.scatter(xsCore, ysCore, s=60, color=colors[cl % len(colors)])
            plt.scatter(xsNonCore, ysNonCore, s=60, facecolor='none', marker='o', linewidth=2, color=colors[cl % len(colors)])

    elif vs == 0:
        xs = outputDf['x']
        ys = outputDf['y']
        plt.scatter(xs, ys, s=60, color='black', marker='x')
    elif vs == 1:
        xsCore = outputDf[outputDf['is_core'] == 1]['x']
        ysCore = outputDf[outputDf['is_core'] == 1]['y']
        xsNonCore = outputDf[outputDf['is_core'] == 0]['x']
        ysNonCore = outputDf[outputDf['is_core'] == 0]['y']
        plt.scatter(xsCore, ysCore, s=60, color=colors[1])
        plt.scatter(xsNonCore, ysNonCore, s=60, facecolor='none', marker='o', linewidth=2, color=colors[0])
    elif vs == 2:
        xs = outputDf['x']
        ys = outputDf['y']
        xsCore = outputDf[outputDf['is_core'] == 1]['x']
        ysCore = outputDf[outputDf['is_core'] == 1]['y']
        xsNonCore = outputDf[outputDf['is_core'] == 0]['x']
        ysNonCore = outputDf[outputDf['is_core'] == 0]['y']

        destXs = []
        destYs = []
        sourceXs = []
        sourceYs = []

        for idx, neighborList in enumerate(dataDct['graph']):
            sourceXs += [ xs[idx] ] * len(neighborList)
            sourceYs += [ ys[idx] ] * len(neighborList)
            destXs += [ xs[neighbor] for neighbor in neighborList ]
            destYs += [ ys[neighbor] for neighbor in neighborList ]
        destXs = np.array(destXs)
        destYs = np.array(destYs)
        sourceXs = np.array(sourceXs)
        sourceYs = np.array(sourceYs)

        plt.quiver(sourceXs, sourceYs, destXs - sourceXs, destYs - sourceYs, angles='xy', scale_units='xy', scale=1, color='lightgrey')
        plt.scatter(xsCore, ysCore, s=60, color=colors[1])
        plt.scatter(xsNonCore, ysNonCore, s=60, facecolor='none', marker='o', linewidth=2, color=colors[0])
      
    
        
    return fig


    #plt.show()

    
for vs in range(0,nVisitedSteps+3):
    fig = createPlot(vs)

    fig.savefig(os.path.join(outputDir, f'andrade_{vs:03.0f}.png'), format='png', bbox_inches='tight')

In [ ]:
set(outputDf['cluster_id'])